# 预测推文是否关于真正的灾害

Kaggle: https://www.kaggle.com/c/nlp-getting-started

# 导入包，读入数据集

In [27]:
import re
import string
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
from torch import nn

In [25]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
label = train['target']
train.drop(['target', 'id'], axis=1, inplace=True)
test.drop(['id'] , axis=1, inplace=True)

data = pd.concat([train, test])
data.head()

,keyword,location,text
0,NaN,NaN,Our Deeds are the Reason of this #earthquake M...
1,NaN,NaN,Forest fire near La Ronge Sask. Canada
2,NaN,NaN,All residents asked to 'shelter in place' are ...
3,NaN,NaN,"13,000 people receive #wildfires evacuation or..."
4,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...


# 数据预处理

## EDA，探索性数据分析
根据[kaggle上的code](https://www.kaggle.com/code/shahules/basic-eda-cleaning-and-glove#Exploratory-Data-Analysis-of-tweets)EDA部分的分析说明，开始进行预处理。

## 数据清洗

做一些基础的清洗，比如去除标点符号、去除html标签以及emoji等。

In [22]:
# removing html tags

def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'', text)

data['text'] = data['text'].apply(remove_html)

In [23]:
# removing emojis
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

data['text'] = data['text'].apply(remove_emoji)

In [26]:
# removing punctuations

def remove_punc(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

data['text'] = data['text'].apply(remove_punc)